# yes24 크롤링

In [279]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys

import json

In [198]:
options = wd.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = wd.Chrome('chromedriver',options=options)

In [202]:
# rank, book_name, authors, publisher, pub_date, sell, price, genre1, genre2,

## test : 링크 내 데이터 가져오기

In [ ]:
url = "https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=279711"
res = requests.get(url)
res.raise_for_status()

soup = BeautifulSoup(res.text, "lxml")

In [252]:
import json
book = soup.find("script")
json_objects = json.loads(book.contents[0])
book_name = json_objects['name']
author = json_objects['author']['name']
author = author.replace(",", " /")
pub_by = json_objects['publisher']['name']
pub_day = json_objects['workExample'][0]['datePublished']
price = int(json_objects['workExample'][0]['potentialAction']['expectsAcceptanceOf']['Price'])
genre1 = soup.select_one('#ulCategory > li:nth-child(1) > a:nth-child(2)').get_text()
genre2 = soup.select_one('#ulCategory > li:nth-child(1) > a:nth-child(3)').get_text()
print(book_name, author, pub_by, pub_day, price, genre1, genre2)

호밀밭의 파수꾼 J.D. 샐린저 지음 / 공경희 옮김 민음사 2001-05-30 7200 소설/시/희곡 영미소설


In [199]:
# https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=MonthlyBest&BranchType=1&CID=0&Year=2001&Month=7&Week=3&page=1&cnt=1000&SortOrder=1
# https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=MonthlyBest&BranchType=1&CID=0&Year=2021&Month=7&Week=3&page=1&cnt=1000&SortOrder=1
# https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=MonthlyBest&BranchType=1&CID=0&Year=2001&Month=7&Week=3&page=1&cnt=1000&SortOrder=1
# https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=MonthlyBest&BranchType=1&CID=170&Year=2000&Month=7&Week=3&page=1&cnt=902&SortOrder=1
# CID (category)
# year
# week(=3)
# month
# page(1,2)


In [312]:
def AladinBookListCrawler(CategoryNum, year_start, year_end, month, page):

    url_1 = 'https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=MonthlyBest&BranchType=1&CID='
    url_2 = '&Year='
    url_3 = '&Month='
    url_4 = '&Week=3&page=' # 월간 판매량은 week3 기준
    url_5 = '&cnt=1000&SortOrder=1'

    book_list = []

    for i in range(year_start, year_end+1) :
        
        rank=0
        
        print("Crawling 시작!")
        print(f"====================>> {i}년")
        
        for j in range(1,page+1):
            
            print(f"====={j}페이지=====")
            
            url_set = url_1 + str(CategoryNum) + url_2 + str(i) + url_3 + str(month) + url_4 + str(j) + url_5
            res = requests.get(url_set)
            res.raise_for_status()
            soup = BeautifulSoup(res.text, "lxml")
            
            books = soup.find_all('a', class_='bo3')
            
            for book in books:
                rank += 1 # 책 순위
                
                book_url = book["href"]
                sub_res = requests.get(book_url)
                sub_res.raise_for_status()
                soup = BeautifulSoup(sub_res.text, "lxml")
                
                book_info = soup.find("script")
                json_objects = json.loads(book_info.contents[0])
                book_name = json_objects['name']
                
                print(f"====={rank}위, {book_name}=====")
                
                author = json_objects['author']['name']
                author = author.replace(",", " /")
                pub_by = json_objects['publisher']['name']
                pub_day = json_objects['workExample'][0]['datePublished']
                price = int(json_objects['workExample'][0]['potentialAction']['expectsAcceptanceOf']['Price'])
                genre1 = soup.select_one('#ulCategory > li:nth-child(1) > a:nth-child(2)').get_text()
                genre2 = soup.select_one('#ulCategory > li:nth-child(1) > a:nth-child(3)').get_text()
                
                book_list.append([i, month, rank, book_name, author, pub_by, pub_day, price, genre1, genre2, book_url])
    
    column_list = ['year', 'month','rank', 'book_name' ,'author', 'pub_by', 'pub_day', 'price', 'genre1', 'genre2', "book_url"]
    
    df = pd.DataFrame(np.array(book_list), columns=column_list)
    
    return df

In [313]:
CategoryNum = 0
year_start = 2001
year_end = 2021
month = 7
page = 2

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36

df = AladinBookListCrawler(CategoryNum, year_start, year_end, month, page)

Crawling 시작!
====================>> 2001년
=====1페이지=====
=====1위, 이윤기의 그리스 로마 신화 1=====
=====2위, 누가 내 치즈를 옮겼을까?=====
=====3위, The Blue Day Book 누구에게나 우울한 날은 있다=====
=====4위, 손님=====
=====5위, 언제까지나 너를 사랑해=====
=====6위, 달님 안녕=====
=====7위, 누가 내 머리에 똥 쌌어?=====
=====8위, 호밀밭의 파수꾼=====
=====9위, TOEIC 답이 보인다=====
=====10위, 너 그거 아니?=====
=====11위, 슈렉!=====
=====12위, 영어의 벽을 뚫는 정찬용식 테이프 1 - 테이프 2개=====
=====13위, 기차 ㄱ ㄴ ㄷ=====
=====14위, 부자 아빠 가난한 아빠 1=====
=====15위, 이문열 이희재 만화 삼국지 1=====
=====16위, 내가 아빠를 얼마나 사랑하는지 아세요?=====
=====17위, Collins Cobuild English Dictionary for Advanced Learners (3판)=====
=====18위, 춘아, 춘아, 옥단춘아, 네 아버지 어디 갔니?=====
=====19위, 신비한 동물 사전=====
=====20위, 예수는 없다=====
=====21위, 퀴디치의 역사=====
=====22위, 사과가 쿵!=====
=====23위, Hackers TOEFL Grammar=====
=====24위, B급 좌파=====
=====25위, 거울=====
=====26위, 아무것도 못 버리는 사람=====
=====27위, 창가의 토토=====
=====28위, 싹싹싹=====
=====29위, 심심해서 그랬어=====
=====30위, 알도와 떠도는 사원 -상=====
=====31위, 강아지똥=====
=====32위, 마르크스 평전=====
=====33위, 아주 작은 차이=====
====

ConnectionError: HTTPSConnectionPool(host='www.aladin.co.kr', port=443): Max retries exceeded with url: /shop/wproduct.aspx?ItemId=2206100 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000023DA10C4E08>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

In [314]:


df.to_csv(f'{year_start}_{year_end}_{CategoryNum}_{month}월_순위.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 사람처럼 보이도록 시간 제한 둘 것,
# 헤더 조정